# Data Exploration: Carbitrage Dataset

This notebook is designed to help you explore the Carbitrage dataset and understand its structure and contents. We'll connect to Google BigQuery, inspect the data schema, perform some basic queries, and identify any potential issues (like missing values or outliers) that might impact our regression modeling.

Make sure you’re connected to BigQuery with the appropriate project permissions before you begin.


## Setting Up BigQuery Client

To interact with BigQuery, we’ll initialize a BigQuery client. Make sure you’re signed into your Google Cloud account, as this notebook uses your active session for authentication.


In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Connect to GBQ
client = bigquery.Client()


## Inspecting the Dataset Schema

Let's start by examining the schema of the `processed_listing_pages` table in the `umt-msba.carbitrage` dataset. This will help us understand the available fields and their data types.


In [3]:
# Define the table reference
table_id = "umt-msba.carbitrage.processed_listing_pages"

# Fetch the table schema
table = client.get_table(table_id)
print("Schema for table {}: \n".format(table_id))
for schema_field in table.schema:
    print(f"{schema_field.name}: {schema_field.field_type}")


Schema for table umt-msba.carbitrage.processed_listing_pages: 

url: STRING
location: STRING
post_id: INTEGER
time_posted: TIMESTAMP
name: STRING
make: STRING
model: STRING
year: INTEGER
odometer: INTEGER
title: STRING
paint: STRING
drive: STRING
cylinders: STRING
fuel: STRING
type: STRING
transmission: STRING
condition: STRING
vin: STRING
price: FLOAT
posting_body_text: STRING
title_text: STRING
num_images: INTEGER
latitude: FLOAT
longitude: FLOAT
basic_processed_time: TIMESTAMP
needs_basic_parsing: BOOLEAN
ai_processed_time: TIMESTAMP
needs_ai_parsing: BOOLEAN
fuzzy_wuzzy_match_score: FLOAT
datetime_raw_html_pulled: DATETIME
mirror_timestamp: TIMESTAMP


## Previewing the Data

Let’s preview the first 10 rows of the table to get a sense of the data and its format.


In [4]:
# Query to preview the data
query = f"""
SELECT url,
    time_posted,
    location,
    make,
    model,
    year,
    title_text, 
    odometer,
    price
FROM `{table_id}`
WHERE make IS NOT NULL AND model IS NOT NULL AND year IS NOT NULL
AND time_posted > "2024-05-01"
ORDER BY time_posted DESC
LIMIT 10
"""

# Run the query and display results
preview_df = client.query(query).to_dataframe()
preview_df.head(10)


,url,time_posted,location,make,model,year,title_text,odometer,price
0,https://boston.craigslist.org/gbs/cto/d/bright...,2024-11-03 14:33:45+00:00,boston,toyota,rav4,2017,2017 Toyota RAV4 Hybrid,73750,19900.0
1,https://boston.craigslist.org/gbs/cto/d/arling...,2024-11-03 14:28:37+00:00,boston,ford,focus,2008,2008 Ford Focus SE Sedan 4D,181000,2000.0
2,https://longisland.craigslist.org/cto/d/amityv...,2024-11-03 14:20:39+00:00,longisland,ford,focus,2013,Ford focus,197000,3000.0
3,https://newjersey.craigslist.org/cto/d/whippan...,2024-11-03 14:18:26+00:00,newjersey,ford,f250,2012,Ford F250 Super Duty 4 X 4,154000,15495.0
4,https://newjersey.craigslist.org/cto/d/ringwoo...,2024-11-03 14:15:04+00:00,newjersey,subaru,crosstrek,2014,2014 Subaru Crosstrek,141000,4600.0
5,https://charlottesville.craigslist.org/cto/d/c...,2024-11-03 14:09:36+00:00,charlottesville,honda,accord,1992,1992 Honda accord Gas sipper,228200,2000.0
6,https://orlando.craigslist.org/cto/d/orlando-2...,2024-11-03 14:07:01+00:00,orlando,land rover,range rover sport,2008,2008 Land Rover Range Rover Sport,156487,5500.0
7,https://tampa.craigslist.org/hil/cto/d/brandon...,2024-11-03 14:02:52+00:00,tampa,mazda,rx-8,2004,‘04 Mazda Rx-8,180000,2100.0
8,https://newjersey.craigslist.org/cto/d/lodi-su...,2024-11-03 14:00:28+00:00,newjersey,subaru,legacy,2011,Subaru legacy 2011 premium,115000,6500.0
9,https://boston.craigslist.org/nos/cto/d/malden...,2024-11-03 13:58:56+00:00,boston,chevrolet,colorado,2012,2012 Chevy Colorado,95445,10000.0


## Basic Descriptive Statistics

To get a better understanding of the numerical fields in the dataset, let's calculate some basic statistics, such as the mean, median, and standard deviation for key variables like `year`, `odometer`, and `price`.


In [5]:
# Query for basic descriptive statistics
query = f"""
SELECT
    MIN(year) AS min_year,
    MAX(year) AS max_year,
    AVG(odometer) AS avg_odometer,
    STDDEV(odometer) AS stddev_odometer,
    AVG(price) AS avg_price,
    STDDEV(price) AS stddev_price
FROM `{table_id}`
WHERE year IS NOT NULL
  AND odometer IS NOT NULL
  AND price IS NOT NULL
  AND time_posted > "2024-05-01"
"""

# Run the query and display results
stats_df = client.query(query).to_dataframe()
stats_df


,min_year,max_year,avg_odometer,stddev_odometer,avg_price,stddev_price
0,1900,2025,142849.070393,255739.442183,11623.736006,13660.925348


## Checking for Missing Values

Next, we'll check the dataset for any missing values. Identifying fields with missing data will help us determine if we need to address these gaps during data cleaning.


In [6]:
# Query to check for missing values in key columns
query = f"""
SELECT
    COUNT(*) AS total_rows,
    COUNTIF(year IS NULL) AS missing_year,
    COUNTIF(odometer IS NULL) AS missing_odometer,
    COUNTIF(price IS NULL) AS missing_price,
    COUNTIF(make IS NULL) AS missing_make,
    COUNTIF(model IS NULL) AS missing_model
FROM `{table_id}`
WHERE time_posted > "2024-05-01"
"""

# Run the query and display results
missing_df = client.query(query).to_dataframe()
missing_df


,total_rows,missing_year,missing_odometer,missing_price,missing_make,missing_model
0,1516083,2934,4361,41650,47310,185793


## Initial Observations and Next Steps

Continue the data exploration, note any observations about the dataset (e.g., ranges, missing values, outliers). These findings will inform your decisions on data cleaning, feature engineering, and model selection for the regression modeling phase.

I'd recommend just pulling down the most recent 6 months or so of data. 